# Motivation
The goal of this project is to gather insight into the SNAP Benefits program while showcasing my data wrangling, statistic calculation, data visualization, and data analysis skills.

# Initial Questions
To guide my project I have decided to answer the following questions.
1. How has the national SNAP participation changed over time?
2. How has SNAP participation changed in each state over time?
3. How has the national average SNAP benefit changed over time?
4. How has the average SNAP benefit in each state changed over time?
5. Which states provide the largest SNAP benefit?
6. Which states have the highest SNAP participation
7. How did each state's SNAP participation change during 2008-2013?
8. How did each state's average SNAP benefit change during 2008-2013? 

# Data Sources
1. SNAP Benefits: **ZIP File: FY 1969 - FY 2021** [https://www.fns.usda.gov/pd/supplemental-nutrition-assistance-program-snap](https://www.fns.usda.gov/pd/supplemental-nutrition-assistance-program-snap) 
2. State Population Data: [Annual Estimates of the Population for the U.S. and States, and for Puerto Rico](https://fred.stlouisfed.org/release?rid=118)
3. US Population Data Monthly: [Population](https://fred.stlouisfed.org/series/POPTHM)
4. US Food Prices by Region [BLS](https://data.bls.gov/cgi-bin/srgate)
    * Series Names and IDS
        * Food at home in Northeast urban, all urban consumers, not seasonally adjusted, CUUR0100SAF11
        * Food at home in Midwest urban, all urban consumers, not seasonally adjusted, CUUR0200SAF11
        * Food at home in South urban, all urban consumers, not seasonally adjusted, CUUR0300SAF11
        * Food at home in West urban, all urban consumers, not seasonally adjusted, CUUR0400SAF11
5. US National Food Prices [Consumer Price Index for All Urban Consumers: Food in U.S. City Average](https://fred.stlouisfed.org/series/CPIUFDNS)
6. Food Regions [https://www.bls.gov/cpi/regional-resources.htm](https://www.bls.gov/cpi/regional-resources.htm)

# Goal of Data Wrangling
The goal of this data wrangling notebook is to take the data from the above sources, clean the data, transform it into a more usable format, and then place all of the data into one excel file across multiple sheets.

In [1]:
import pandas as pd
import xlwings as xw
from tqdm.notebook import tqdm

### Necessary Information
I used pyautogui to automate typing out many of the lists below. In the ZIP file for SNAP Benefits there is a change in the format of the excel tables starting with the file FY15.xls. To account for this change I have new column names and old column names, new file paths and old file paths.

In [151]:
column_names_old = ['dates','Household Participation','Individual Participation','Benefit Per Household','Benefit Per Person','Cost of Benefit']
column_names_new = ['dates','Household Participation','Individual Participation','Cost of Benefit','Benefit Per Household','Benefit Per Person']
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming",'US Population']
old_sheet_names = ['NERO','MARO','SERO','MWRO','SWRO ','MPRO','WRO','US Summary ']
new_sheet_names = ['NERO','MARO','SERO','MWRO','SWRO','MPRO','WRO','US Summary']
old_file_paths = [r'archive\SNAPZip69throughCurrent-1\FY89.xls',
r'SNAPZip69throughCurrent-1\FY90.xls',
r'SNAPZip69throughCurrent-1\FY91.xls',
r'SNAPZip69throughCurrent-1\FY92.xls',
r'SNAPZip69throughCurrent-1\FY93.xls',
r'SNAPZip69throughCurrent-1\FY94.xls',
r'SNAPZip69throughCurrent-1\FY95.xls',
r'SNAPZip69throughCurrent-1\FY96.xls',
r'SNAPZip69throughCurrent-1\FY97.xls',
r'SNAPZip69throughCurrent-1\FY98.xls',
r'SNAPZip69throughCurrent-1\FY99.xls',
r'SNAPZip69throughCurrent-1\FY00.xls',
r'SNAPZip69throughCurrent-1\FY01.xls',
r'SNAPZip69throughCurrent-1\FY02.xls',
r'SNAPZip69throughCurrent-1\FY03.xls',
r'SNAPZip69throughCurrent-1\FY04.xls',
r'SNAPZip69throughCurrent-1\FY05.xls',
r'SNAPZip69throughCurrent-1\FY06.xls',
r'SNAPZip69throughCurrent-1\FY07.xls',
r'SNAPZip69throughCurrent-1\FY08.xls',
r'SNAPZip69throughCurrent-1\FY09.xls',
r'SNAPZip69throughCurrent-1\FY10.xls',
r'SNAPZip69throughCurrent-1\FY11.xls',
r'SNAPZip69throughCurrent-1\FY12.xls',
r'SNAPZip69throughCurrent-1\FY13.xls',
r'SNAPZip69throughCurrent-1\FY14.xls']

new_file_paths = [r'SNAPZip69throughCurrent-1\FY15.xls',
r'SNAPZip69throughCurrent-1\FY16.xls',
r'SNAPZip69throughCurrent-1\FY17.xls',
r'SNAPZip69throughCurrent-1\FY18.xls',
r'SNAPZip69throughCurrent-1\FY19.xls',]

### Pull data from each of the above files and place in pandas series
The below code opens up each excel file, scans through each sheet, and finds tables that correspond to each state. Once I have a dataframe representing one of these tables for a state it is placed into a pandas series called data_series. data_series has an index comprised of the 50 states and each value of the series is a dataframe containing data for that state. Each time I open up a new file I update the dataframes within data_series with the new data.

In [92]:
data_series = pd.Series()
i = 0
for file in tqdm(old_file_paths):
    excel_app = xw.App(visible=False)
    book = excel_app.books.open(file)
    for sheet_name in old_sheet_names:
        sheet = book.sheets[sheet_name]
        data_df = sheet.range('A5:F300').options(pd.DataFrame).value
        data_df.reset_index(inplace = True)
        data_df.columns = column_names_old
        data_df = data_df.loc[data_df.dates.notna(),:]
        data_df.index = range(len(data_df.index))
        states_indices = data_df.dates.loc[data_df.dates.isin(states)]
        states_monthly_data = pd.Series([data_df.iloc[i+1:i+13,:].set_index('dates') for i in states_indices.index],index = states_indices.tolist())
        if i ==0:
            data_series = data_series.append(states_monthly_data)
        else:
            for state in states_monthly_data.index:
                current_df = data_series[state]
                new_df = current_df.append(states_monthly_data.loc[state])
                data_series[state] = new_df
    i=1
    book.close()
    excel_app.quit()
    
for file in tqdm(new_file_paths):
    excel_app = xw.App(visible=False)
    book = excel_app.books.open(file)
    for sheet_name in new_sheet_names:
        sheet = book.sheets[sheet_name]
        data_df = sheet.range('A8:F300').options(pd.DataFrame).value
        data_df.reset_index(inplace = True)
        data_df.columns = column_names_new
        data_df = data_df.loc[data_df.dates.notna(),:]
        data_df.index = range(len(data_df.index))
        states_indices = data_df.dates.loc[data_df.dates.isin(states)]
        states_monthly_data = pd.Series([data_df.iloc[i+1:i+13,:].set_index('dates') for i in states_indices.index],index = states_indices.tolist())
        for state in states_monthly_data.index:
            current_df = data_series[state]
            new_df = current_df.append(states_monthly_data.loc[state])
            data_series[state] = new_df
    book.close()
    excel_app.quit()
    

<ipython-input-92-03a4643d7600>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data_series = pd.Series()


### Save data_series to h5 file
I decided to save the data_series to an h5 file incase I needed to return to the end of the prior step. I then reload this data below.

In [21]:
data_series = pd.read_hdf(r"archive\SNAPZip69throughCurrent-1\Cleaned Data.h5")
data_series = data_series.sort_index()

### Write data_series to excel file
I then write data_series to an excel file where each sheet represents a state. I like to make copies of data throughout the data wrangling process that way I can return to a step if needed.

In [22]:
excel_app = xw.App(visible=True)
book = excel_app.books.open(r"archive\SNAPZip69throughCurrent-1\Cleaned Data.xlsx")
for i,state in enumerate(data_series.index[::-1]):
    if i==0:
        sheet = book.sheets[0]
        sheet.name = state
        sheet.range('A1').options(pd.DataFrame,expand = 'table').value = data_series[state]
    else:
        sheet = book.sheets.add(state)
        sheet.range('A1').options(pd.DataFrame,expand = 'table').value = data_series[state]

### Pull National Level Monthly Data from Files
The next block of code is used to pull national level monthly data from the ``1969-88  National Level Only--State Level Not Available`` file in a similar manner as the state level data was pulled from each file. This data is then pieced together and placed into the ``US Summary`` tab of the excel file created in the last step.

In [58]:
book = xw.Book(r"archive\SNAPZip69throughCurrent-1\1969-88  National Level Only--State Level Not Available.xls")
sheet = book.sheets[0]

data = sheet.range('A10:D202').options(pd.DataFrame).value

data = data.loc[data.index.notna(),:]
data = data.loc[~data.index.str.contains('FY'),:]
data = data.loc[~data.index.str.contains('Transition'),:]
data = data.loc[~data.index.str.contains('Total'),:]

months = ['January','February','March','April','May','June','July','August','September','October','November','December']
years = [1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979]

yearmonth = []
for year in years:
    for month in months:
        date = month+' '+str(year)
        yearmonth.append(date)

yearmonth = yearmonth[6:-3]

data.index = yearmonth

data.columns = ['Individual Participation','Cost of Benefit','Benefit Per Person']

sheet = book.sheets[1]

data1 = sheet.range('A9:F159').options(pd.DataFrame).value

data1 = data1.loc[data1.index.notna(),:]
data1 = data1.loc[~data1.index.str.contains('FY'),:]
data1 = data1.loc[~data1.index.str.contains('---'),:]
data1 = data1.loc[~data1.index.str.contains('Transition'),:]
data1 = data1.loc[~data1.index.str.contains('TOTAL'),:]

months = ['January','February','March','April','May','June','July','August','September','October','November','December']
years = [1979,1980,1981,1982,1983,1984,1985,1986,1987,1988]

yearmonth = []
for year in years:
    for month in months:
        date = month+' '+str(year)
        yearmonth.append(date)

yearmonth = yearmonth[9:-3]

data1.index = yearmonth

data1.columns = ['Individual Participation','Household Participation','Cost of Benefit','Benefit Per Person','Benefit Per Household']

final_data = data.append(data1)

final_data.index.name = 'dates'

final_data.index = pd.to_datetime(final_data.index)

df = pd.read_excel(r"archive\SNAPZip69throughCurrent-1\Cleaned Data.xlsx",sheet_name = 'US Summary',index_col = 'dates')
df.index = pd.to_datetime(df.index)

final_data = final_data.append(df)

book = xw.Book(r"archive\SNAPZip69throughCurrent-1\Cleaned Data.xlsx")
sheet = book.sheets['US Summary']

sheet.range('A1').options(pd.DataFrame,expand = 'table').value = final_data

### Transform Data into More Usable Format
The next block of code takes the data from the ``Cleaned Data`` file which was created two blocks of code ago and transforms it into a more usable format. The final format of this new file is an excel file where each sheet represents one of the SNAP statistics i.e. ``Household Participation``. Within the sheet is a table where each column is a state and each row is a time.

In [111]:
book = xw.Book(r"archive\SNAPZip69throughCurrent-1\Cleaned Data.xlsx")

HP = pd.DataFrame()
IP = pd.DataFrame()
BPH = pd.DataFrame()
BPP = pd.DataFrame()
CB = pd.DataFrame()

for sheet in book.sheets:
    data = sheet.range('A1').options(pd.DataFrame,expand = 'table').value
    name = sheet.name
    HP.loc[:,name] = data.loc[:,'Household Participation']
    IP.loc[:,name] = data.loc[:,'Individual Participation']
    BPH.loc[:,name] = data.loc[:,'Benefit Per Household']
    BPP.loc[:,name] = data.loc[:,'Benefit Per Person']
    CB.loc[:,name] = data.loc[:,'Cost of Benefit']

dfs = [HP,IP,BPH,BPP,CB]
sheet_names = ['Household Partcipation','Individual Participation','Benefit Per Household','Benefit Per Person','Cost of Benefit']
book = xw.Book(r"archive\SNAPZip69throughCurrent-1\Data by Measure.xlsx")
for i,sheet in enumerate(sheet_names):
    sht = book.sheets[sheet]
    sht.range('A1').options(pd.DataFrame,expand = 'table').value = dfs[i]
    

### Clean and Insert Annual State Population Data into New File
The next block of code pulls data from the annual state populatios file. Cleans the columns so they have state names rather than abbreviations and places the dataframe into a new excel file.

In [129]:
state_pops = pd.read_excel(r"archive\SNAPZip69throughCurrent-1\State_Populations.xls",index_col = 'DATE')
state_pops = state_pops*1000

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
us_state_abbrev = {value:key for key,value in us_state_abbrev.items()}

new_cols = []
for col in state_pops.columns:
    abbrev = col[:2]
    name = us_state_abbrev[abbrev]
    new_cols.append(name)

state_pops.columns = new_cols
state_pops.index.name = 'date'

state_pops.to_excel(r"archive\SNAPZip69throughCurrent-1\State_Populations.xlsx")

### Manually Place Remaining Population Data into Data by Measure File
The next step is to place the remaining National Monthly Population data into a new sheet in the Data by Measure file. Along with this population data I also placed the full National Monthly SNAP Benefits data into this sheet.

### Pull Data From Food Prices Excel Files, Clean, and place into Data by Measure File
The next step is to pull data from the food price excel files, tranform each of these tables into a single column and then merge all of the columns together to form a single dataframe. Then we place this final dataframe into the data by measure file

In [39]:
file_paths = [r'archive/Food at home in Midwest urban, all urban consumers, not seasonally adjusted.xlsx',
             r'archive/Food at home in Northeast urban, all urban consumers, not seasonally adjusted.xlsx',
             r'archive/Food at home in West urban, all urban consumers, not seasonally adjusted.xlsx',
             r'archive/Food at home in South urban, all urban consumers, not seasonally adjusted.xlsx']
regions = ['Midwest','Northeast','West','South']

In [40]:
final_df = pd.DataFrame()
for i,file in enumerate(file_paths):
    book = xw.Book(file)
    sheet = book.sheets[0]
    df = sheet.range('A12:P44').options(pd.DataFrame).value
    df = df.iloc[:,:12]

    data = {}
    for year in df.index:
        year = int(year)
        for month in df.loc[year,:].index:
            value = df.loc[year,month]
            ind = month+' '+str(year)
            data[ind] = value
    data = pd.Series(data).iloc[9:-3]
    data.index.name = regions[i]
    final_df.loc[:,data.index.name] = data
final_df.index.name = 'date'

In [44]:
book = xw.Book('archive/Clean Data/Data by Measure.xlsx')
sheet = book.sheets.add('Food Prices by Region')

In [45]:
sheet.range('A1').options(pd.DataFrame,expand = 'table').value = final_df

### Manually Insert National Level Average Food Prices into Data by Measure file.
The enxt step is to manually add the national level average food price data into the data by measure file in the US Data sheet.